## Consumer Food Demand Estimation

Team members: Stone Yan, Angela Chen, Yingyin Li, Tia Pappas, Daniela Salinas, Lyuheng(Kasper) Zheng

#### Our Topic and Population of Interest:

We are interested in the population of Panama. Our topic is about understanding diets in Panama, characterizing the relationship between people’s food choices, budget and prices. 

#### Our Goals:  

1. Analyze Panama households' food choices and expenditures to assess their diets' nutritional content and adequacy.
<br>
2. Estimate food demand among Panama, considering household needs and economic factors, and to analyze how changes in expenditure and food prices influence dietary choices.
<br>
3. Conduct counterfactual experiments to assess how hypothetical economic scenarios, such as changes in food expenditure, prices, shortages, and income, could affect Panama's nutrition.


### (A) Choice of Population & Supporting Data

#### Data Frame Set Up

In the following we will be estimating Constant Frisch Elasticity (CFE) demand systems for our selected population - Panama. We first installed our prerequisites. 

In [1]:
!pip install -r requirements.txt 
!pip install CFEDemands --upgrade --pre -q
!pip install eep153_tools --upgrade -q

In the following we will be estimating Constant Frisch Elasticity (CFE) demand systems for our selected population - Panama. We first imported the data we need from the Google Sheet. 

In [2]:
InputFiles = {'Food Expenditures':('1gcAb2jlGQNrD2zrrTEbjL47vbXoxCHkkjHSYzD0-Tiw','Food Expenditures'),
              'Food Prices':('1gcAb2jlGQNrD2zrrTEbjL47vbXoxCHkkjHSYzD0-Tiw','Food Prices'),
              'Household Characteristics':('1gcAb2jlGQNrD2zrrTEbjL47vbXoxCHkkjHSYzD0-Tiw','Household Characteristics'),
              'FCT':('1gcAb2jlGQNrD2zrrTEbjL47vbXoxCHkkjHSYzD0-Tiw','FCT'),
              'Copy of RDI':('1gcAb2jlGQNrD2zrrTEbjL47vbXoxCHkkjHSYzD0-Tiw','Copy of RDI'),}

We then imported packages that are needed. 

In [3]:
import numpy as np
import pandas as pd
from eep153_tools.sheets import read_sheets
import cfe
from cfe.estimation import drop_columns_wo_covariance
from cfe import Regression

In [4]:
def get_clean_sheet(key,sheet=None):

    df = read_sheets(key,sheet=sheet)
    df.columns = [c.strip() for c in df.columns.tolist()]

    df = df.loc[:,~df.columns.duplicated(keep='first')]   

    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)

    df = df.loc[~df.index.duplicated(), :]

    return df

In [5]:
# Get food expenditures 
expenditures = get_clean_sheet(InputFiles['Food Expenditures'][0],
                    sheet=InputFiles['Food Expenditures'][1])

if 'm' not in expenditures.columns:
    expenditures['m'] = 1

expenditures = expenditures.set_index(['i','t','m'])
expenditures.columns.name = 'j'

expenditures = expenditures.apply(lambda x: pd.to_numeric(x,errors='coerce'))
expenditures = expenditures.replace(0,np.nan)

In [6]:
expenditures = expenditures.loc[~expenditures.index.duplicated()]

In [7]:
# Get Household characteristics 
hh_characteristics = get_clean_sheet(InputFiles['Household Characteristics'][0],
                    sheet=InputFiles['Household Characteristics'][1])

if 'm' not in hh_characteristics.columns:
    hh_characteristics['m'] = 1

hh_characteristics = hh_characteristics.set_index(['i','t','m'])
hh_characteristics.columns.name = 'k'
#hh_characteristics.name = 'value'  

hh_characteristics = hh_characteristics.apply(lambda x: pd.to_numeric(x,errors='coerce'))

In [8]:
hh_characteristics.index.is_unique 

True

In [9]:
# Get food prices and call it p
url = 'https://docs.google.com/spreadsheets/d/1gcAb2jlGQNrD2zrrTEbjL47vbXoxCHkkjHSYzD0-Tiw/edit#gid=2085637103'
p = read_sheets(url,sheet='Food Prices',nheaders=2)

p.columns.names = ['t','m']
p.groupby(level='j').mean()

#use tranpose to switch columns and rows
p = p.transpose()
if 'm' not in p.columns:
    p['m']=1

p=p.apply(lambda x: pd.to_numeric(x,errors='coerce'))
p=p.replace(0,np.nan)

In [10]:
# Get FCT
fct = get_clean_sheet(InputFiles['FCT'][0],
                    sheet=InputFiles['FCT'][1])

fct = fct.set_index('j')
fct.columns.name = 'n'

fct = fct.apply(lambda x: pd.to_numeric(x,errors='coerce'))

In [11]:
# Get RDI
rdi = get_clean_sheet(InputFiles['Copy of RDI'][0],
                    sheet=InputFiles['Copy of RDI'][1])
rdi = rdi.set_index('n')
rdi.columns.name = 'k'

#### Show Organized Data Frame

Show food expenditures dataframe.

In [12]:
expenditures.head()

,,j,Aceite Vegetal,Aguacates,Ahí Verde,Ajo,Alimento Infantil,Apio,Arroz,Arvejas,Azúcar,Café Y Té,...,"Sodas, Refrescos Y Jugos",Sopa Enlatada,Tercer otro,Tomate,Viscera De Res,Visceras De Pollo O Gallina,Yuca,Zanahoria,Zapallo / Chayote,Ñame
i,t,m,,,,,,,,,,,,,,,,,,,,,
19971000,1997,Chíriqui,6.0,NaN,0.5,NaN,2.7,0.3,7.25,NaN,1.6,NaN,...,8.0,0.7,NaN,NaN,NaN,NaN,NaN,0.8,NaN,0.4
19971001,1997,Chíriqui,NaN,NaN,NaN,NaN,NaN,NaN,4.80,NaN,NaN,1.4,...,1.5,NaN,NaN,0.3,NaN,NaN,NaN,NaN,NaN,NaN
19971002,1997,Chíriqui,3.5,NaN,NaN,NaN,NaN,NaN,6.00,NaN,2.0,3.6,...,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
19971003,1997,Chíriqui,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19971005,1997,Chíriqui,3.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,1.5,...,NaN,0.7,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN


Show household characteristics dataframe.

In [13]:
hh_characteristics.head()

,,k,women,boys,men,girls,Males 00-03,Females 00-03,Males 04-08,Females 04-08,Males 09-13,Females 09-13,Males 14-18,Females 14-18,Males 19-30,Females 19-30,Males 31-50,Females 31-50,Males 51-99,Females 51-99,log HSize
i,t,m,,,,,,,,,,,,,,,,,,,
1997125,1997,Herrera,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1.098612
1997160,1997,Herrera,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1.098612
1997170,1997,Bocas Del Toro,1,3,1,4,1,2,1,0,0,1,1,1,0,0,0,1,1,0,2.197225
1997177,1997,Bocas Del Toro,4,9,1,5,2,1,4,1,1,3,2,0,0,2,0,2,1,0,2.944439
1997178,1997,Bocas Del Toro,2,1,2,2,0,0,0,1,1,0,0,1,2,0,0,1,0,1,1.945910


Show food prices dataframe.

In [14]:
p.head()

j                   Aceite Vegetal                                            \
u                       1/2 gallon  None bottle    can gallon gram large bay   
t    m                                                                         
1997 Bocas Del Toro          3.225   NaN   1.20  2.975   6.70  NaN     1.500   
     Chíriqui                3.250  0.10   1.35    NaN   5.95  NaN     1.350   
     Coclé                   2.990  0.10   1.30    NaN   5.90  NaN     1.375   
     Colón                   3.250  0.10   1.34    NaN   5.75  NaN       NaN   
     Darién                  3.200  0.25   1.40    NaN   4.85  NaN     1.525   

j                                                ... Zapallo / Chayote  \
u                   liter medium bar milliliter  ...              unit   
t    m                                           ...                     
1997 Bocas Del Toro   NaN      1.050        NaN  ...              0.10   
     Chíriqui         NaN      1.325        NaN  ...              0.10   
     Coclé            NaN      1.275        NaN  ...              0.15   
     Colón            NaN      0.850        NaN  ...              0.25   
     Darién           NaN      1.400        NaN  ...               NaN   

j                                    Ñame                                 \
u                       value bag or sack gram ounce package pound  unit   
t    m                                                                     
1997 Bocas Del Toro       NaN         3.6  NaN   NaN     NaN  0.40   NaN   
     Chíriqui        0.084416         NaN  NaN   NaN     NaN  0.40  0.15   
     Coclé           0.201299         NaN  NaN   NaN     NaN  0.50  0.50   
     Colón           0.045455         NaN  NaN   NaN     NaN  0.40  0.60   
     Darién               NaN         NaN  NaN   NaN     NaN  0.15   NaN   

j                              m  
u                       value     
t    m                            
1997 Bocas Del Toro       NaN  1  
     Chíriqui        0.038961  1  
     Coclé                NaN  1  
     Colón           0.064935  1  
     Darién               NaN  1  

[5 rows x 691 columns]

Show FCT dataframe.

In [15]:
fct.head()

n,Agua,Energía,Proteina,Grasa Total,Carbo-hidratos,Fibra Diet. total,Ceniza,Calcio,Fosforo,Hierro,...,Colesterol,Potasio,Sodio,Zinc,Magnesio,Vit. B6,Vit. B12,Ac. Fólico,Folato Equiv. FD,Fracción Comestible
j,,,,,,,,,,,,,,,,,,,,,
Aceite Vegetal,0.00,884,0.00,100.00,0.00,0.0,0.00,0,0,0.00,...,0,0,0,0.00,0,0.00,0.0,0,0,1.00
Aguacates,73.23,160,2.00,14.66,8.53,6.7,1.58,12,52,0.55,...,0,485,7,0.64,29,0.26,0.0,0,81,0.74
Ahí Verde,93.89,20,0.86,0.17,3.64,1.7,0.43,10,20,0.34,...,0,175,3,0.13,10,0.22,0.0,0,11,0.82
Ajo,63.80,134,5.30,0.20,29.30,0.0,1.40,38,134,1.40,...,0,529,19,1.00,0,0.00,0.0,3,0,0.94
Alimento Infantil,3.00,510,12.50,27.00,54.30,0.0,3.20,530,420,9.00,...,0,600,180,6.00,55,0.30,1.5,80,136,1.00


Show RDI dataframe.

In [16]:
rdi.head()

k,F 00-03,M 00-03,F 04-08,M 04-08,F 09-13,M 09-13,F 14-18,M 14-18,F 19-30,M 19-30,F 31-50,M 31-50,F 51+,M 51+
n,,,,,,,,,,,,,,
Energy,1000.0,1000.0,1200.0,1400.0,1600.0,1800.0,1800.0,2200.0,2000.0,2400.0,1800.0,2200.0,1600.0,2000.0
Protein,13.0,13.0,19.0,19.0,34.0,34.0,46.0,52.0,46.0,56.0,46.0,56.0,46.0,56.0
Fiber,14.0,14.0,16.8,19.6,22.4,25.2,25.2,30.8,28.0,33.6,25.2,30.8,22.4,28.0
Folate,150.0,150.0,200.0,200.0,300.0,300.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
Calcium,700.0,700.0,1000.0,1000.0,1300.0,1300.0,1300.0,1300.0,1000.0,1000.0,1000.0,1000.0,1200.0,1000.0


### (A) Estimate demand system

 Let $y_{i}^j$ be log household expenditure on food item $j$ from household $i$ of Panama. Our estimation regression takes the following form: 
 $$
      y^j_{i} = A^j(p) + \gamma_j'd_i + \beta_j w_i + \zeta^j_i.
$$

The formula above models the log household expenditure as a function of :<br>

$A^j(p)$: A price index for food $j$, capturing how the pricing of good $j$ affects expenditure on food $j$;
<br>
$\gamma_j'd_i$: A household characterics demonstrating how demographics affects expenditure on food $j$; $\gamma_j$ is its coefficient.
<br>
$\beta_j w_i$: This term captures how the household's overall wealth affects its expenditure on food $j$; $\beta_j$ is its coefficent.
<br>
$\zeta^j_i$: This term captures other unobserved effect that influence food expenditure. 


In [17]:
# log_expenditures represents the logarithm of food expenditures.
log_expenditures = np.log(expenditures)

log_expenditures.head()

,,j,Aceite Vegetal,Aguacates,Ahí Verde,Ajo,Alimento Infantil,Apio,Arroz,Arvejas,Azúcar,Café Y Té,...,"Sodas, Refrescos Y Jugos",Sopa Enlatada,Tercer otro,Tomate,Viscera De Res,Visceras De Pollo O Gallina,Yuca,Zanahoria,Zapallo / Chayote,Ñame
i,t,m,,,,,,,,,,,,,,,,,,,,,
19971000,1997,Chíriqui,1.791759,NaN,-0.693147,NaN,0.993252,-1.203973,1.981001,NaN,0.470004,NaN,...,2.079442,-0.356675,NaN,NaN,NaN,NaN,NaN,-0.223144,NaN,-0.916291
19971001,1997,Chíriqui,NaN,NaN,NaN,NaN,NaN,NaN,1.568616,NaN,NaN,0.336472,...,0.405465,NaN,NaN,-1.203973,NaN,NaN,NaN,NaN,NaN,NaN
19971002,1997,Chíriqui,1.252763,NaN,NaN,NaN,NaN,NaN,1.791759,NaN,0.693147,1.280934,...,NaN,1.252763,NaN,NaN,NaN,NaN,NaN,NaN,0.693147,NaN
19971003,1997,Chíriqui,1.252763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19971005,1997,Chíriqui,1.308333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.504077,0.405465,...,NaN,-0.356675,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN


In [18]:
# use is the intersection of indexs between log_expenditures and household characteristics.
use = log_expenditures.index.intersection(hh_characteristics.index)

log_expenditures = log_expenditures.loc[use,:]
hh_characteristics = hh_characteristics.loc[use,:]

In [19]:
log_expenditures = log_expenditures.stack()
hh_characteristics = hh_characteristics.stack()

# Check that indices are in right places!
assert log_expenditures.index.names == ['i','t','m','j']
assert hh_characteristics.index.names == ['i','t','m','k']

#### 1. Basic Estimation Using Regression

In [20]:
#set up regression
result = Regression(y = log_expenditures,d = hh_characteristics)

In [ ]:
#get regression results
result.predicted_expenditures()

In [ ]:
# use the graph to compare log food expenditures and actual expenditures
%matplotlib widget
df = pd.DataFrame({'y':log_expenditures,'yhat':result.get_predicted_log_expenditures()})
df.plot.scatter(x='yhat',y='y')

#### 2. Analyze Income Elasticity

In [ ]:
#get the value of beta - As shown above, beta captures how the household's overall wealth affects its expenditures on food. 
result.get_beta().sort_values()

In [ ]:
#graph beta
result.graph_beta()

In [ ]:
#get the value of gamma - As shown above, gamma captures how household characteristics affects its expenditures on food. 
result.gamma

In [ ]:
#To save result
result.to_pickle('estimates.pickle')

result = cfe.regression.read_pickle('estimates.pickle')